In [1]:
!pip install pyaurn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
import tensorflow as tf
import pandas as pd
import requests
import pyaurn

from google.colab import drive
from shutil import make_archive, move

In [3]:
metadata = pyaurn.importMeta().set_index("site_id")

In [4]:
display(metadata)
filtered_metadata = metadata.drop(["site_name", "location_type", "parameter", "Parameter_name", "start_date", "end_date", "ratified_to", "zone", "agglomeration", "local_authority"], axis=1)
site = filtered_metadata.loc[["WREX"]]
display(site)

,site_name,location_type,latitude,longitude,parameter,Parameter_name,start_date,end_date,ratified_to,zone,agglomeration,local_authority
site_id,,,,,,,,,,,,
ABD,Aberdeen,Urban Background,57.157360,-2.094278,O3,Ozone,2003-08-01,2021-09-20,2021-09-20,North East Scotland,NaN,Aberdeen City
ABD9,Aberdeen Erroll Park,Urban Background,57.157400,-2.094770,O3,Ozone,2021-10-01,ongoing,2022-06-30,North East Scotland,NaN,Aberdeen City
ABD7,Aberdeen Union Street Roadside,Urban Traffic,57.144555,-2.106472,NO,Nitric oxide,2008-01-01,ongoing,2022-06-30,North East Scotland,NaN,Aberdeen City
ABD8,Aberdeen Wellington Road,Urban Traffic,57.133888,-2.094198,NO,Nitric oxide,2016-02-09,ongoing,2022-06-30,North East Scotland,NaN,Aberdeen City
ARM6,Armagh Roadside,Urban Traffic,54.353728,-6.654558,NO,Nitric oxide,2009-01-01,ongoing,2022-06-30,Northern Ireland,NaN,Armagh
...,...,...,...,...,...,...,...,...,...,...,...,...
WRAY,Wray,Rural Background,54.104666,-2.584182,O3,Ozone,1985-04-01,1988-02-29,1988-02-29,North West & Merseyside,NaN,Lancaster
WREX,Wrexham,Urban Traffic,53.042282,-3.002829,NO,Nitric oxide,2002-03-06,ongoing,2022-06-30,North Wales,NaN,Wrexham
YW,Yarner Wood,Rural Background,50.597600,-3.716510,O3,Ozone,1987-06-26,ongoing,2022-06-30,South West,NaN,Teignbridge


,latitude,longitude
site_id,,
WREX,53.042282,-3.002829


In [5]:
data = pyaurn.importAURN("WREX", range(2010, 2022)).reset_index()

data["site_id"] = "WREX"
display(data)

,date,NO,NO2,NOXasNO2,SO2,wd,ws,temp,site,code,PM10,PM2.5,site_id
0,2010-01-01 00:00:00,3.00000,36.00000,40.00000,3.00000,NaN,NaN,NaN,Wrexham,WREX,NaN,NaN,WREX
1,2010-01-01 01:00:00,3.00000,27.00000,31.00000,5.00000,NaN,NaN,NaN,Wrexham,WREX,NaN,NaN,WREX
2,2010-01-01 02:00:00,4.00000,25.00000,31.00000,3.00000,NaN,NaN,NaN,Wrexham,WREX,NaN,NaN,WREX
3,2010-01-01 03:00:00,3.00000,19.00000,23.00000,3.00000,NaN,NaN,NaN,Wrexham,WREX,NaN,NaN,WREX
4,2010-01-01 04:00:00,3.00000,19.00000,23.00000,3.00000,NaN,NaN,NaN,Wrexham,WREX,NaN,NaN,WREX
...,...,...,...,...,...,...,...,...,...,...,...,...,...
105187,2021-12-31 19:00:00,2.13558,4.26269,7.53720,0.35230,229.4,7.5,9.5,Wrexham,WREX,10.575,6.698,WREX
105188,2021-12-31 20:00:00,1.91840,5.03582,7.97732,0.28184,219.7,7.6,9.7,Wrexham,WREX,10.550,6.651,WREX
105189,2021-12-31 21:00:00,2.71472,7.17080,11.33330,0.21138,218.5,8.8,10.5,Wrexham,WREX,9.200,6.226,WREX
105190,2021-12-31 22:00:00,1.44785,4.98710,7.20710,0.07046,215.3,9.8,11.0,Wrexham,WREX,10.400,7.665,WREX


In [6]:
data["date"] = data["date"].apply(lambda e: e.timestamp())
filtered_data = data[["date", "PM2.5", "site_id"]].dropna()
display(filtered_data)

,date,PM2.5,site_id
76681,1.538356e+09,5.354,WREX
76682,1.538359e+09,4.175,WREX
76683,1.538363e+09,3.373,WREX
76684,1.538366e+09,3.160,WREX
76685,1.538370e+09,3.962,WREX
...,...,...,...
105187,1.640977e+09,6.698,WREX
105188,1.640981e+09,6.651,WREX
105189,1.640984e+09,6.226,WREX
105190,1.640988e+09,7.665,WREX


In [7]:
merged_data = filtered_data.merge(site, on="site_id").drop("site_id", axis=1)
display(merged_data)

,date,PM2.5,latitude,longitude
0,1.538356e+09,5.354,53.042282,-3.002829
1,1.538359e+09,4.175,53.042282,-3.002829
2,1.538363e+09,3.373,53.042282,-3.002829
3,1.538366e+09,3.160,53.042282,-3.002829
4,1.538370e+09,3.962,53.042282,-3.002829
...,...,...,...,...
25891,1.640977e+09,6.698,53.042282,-3.002829
25892,1.640981e+09,6.651,53.042282,-3.002829
25893,1.640984e+09,6.226,53.042282,-3.002829
25894,1.640988e+09,7.665,53.042282,-3.002829


In [8]:
train_dataset = merged_data.sample(frac=0.8)
test_dataset = merged_data.drop(train_dataset.index)
display(train_dataset)
display(test_dataset)

,date,PM2.5,latitude,longitude
12682,1.593299e+09,1.627,53.042282,-3.002829
24585,1.636272e+09,9.623,53.042282,-3.002829
25612,1.639973e+09,5.967,53.042282,-3.002829
23265,1.631520e+09,7.146,53.042282,-3.002829
8941,1.579799e+09,13.609,53.042282,-3.002829
...,...,...,...,...
8808,1.579320e+09,6.203,53.042282,-3.002829
17840,1.611968e+09,8.326,53.042282,-3.002829
6521,1.571087e+09,10.165,53.042282,-3.002829
20494,1.621534e+09,1.958,53.042282,-3.002829


,date,PM2.5,latitude,longitude
2,1.538363e+09,3.373,53.042282,-3.002829
5,1.538374e+09,4.198,53.042282,-3.002829
8,1.538384e+09,5.189,53.042282,-3.002829
32,1.538471e+09,1.368,53.042282,-3.002829
35,1.538482e+09,1.439,53.042282,-3.002829
...,...,...,...,...
25873,1.640912e+09,3.137,53.042282,-3.002829
25879,1.640934e+09,3.892,53.042282,-3.002829
25881,1.640941e+09,5.047,53.042282,-3.002829
25887,1.640963e+09,6.226,53.042282,-3.002829


In [9]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop("PM2.5")
test_labels = test_features.pop("PM2.5")
train_features.shape

(20717, 3)

In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(3,)),
    #categoryLayer,
    tf.keras.layers.Dense(64),
    tf.keras.layers.Dense(64),
    tf.keras.layers.Dense(1)
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                256       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 4,481
Trainable params: 4,481
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    loss="mean_absolute_error"
)

In [13]:
print(train_features.dtypes)
history = model.fit(
    train_features,
    train_labels,
    epochs=10,
    validation_data = (test_features, test_labels)
)

date         float64
latitude     float64
longitude    float64
dtype: object
Epoch 1/10
648/648 [==============================] - 2s 2ms/step - loss: 104994816.0000 - val_loss: 8391.5029
Epoch 2/10
648/648 [==============================] - 1s 2ms/step - loss: 289.3140 - val_loss: 7.2282
Epoch 3/10
648/648 [==============================] - 1s 2ms/step - loss: 4.7905 - val_loss: 4.3590
Epoch 4/10
648/648 [==============================] - 1s 2ms/step - loss: 4.5938 - val_loss: 5.0899
Epoch 5/10
648/648 [==============================] - 1s 2ms/step - loss: 5.0090 - val_loss: 4.5031
Epoch 6/10
648/648 [==============================] - 1s 2ms/step - loss: 3285485824.0000 - val_loss: 304125.9375
Epoch 7/10
648/648 [==============================] - 1s 2ms/step - loss: 26008.3438 - val_loss: 4622.4497
Epoch 8/10
648/648 [==============================] - 1s 2ms/step - loss: 1859.2701 - val_loss: 1519.0953
Epoch 9/10
648/648 [==============================] - 1s 2ms/step - loss: 1360.0022

In [14]:
display(model.predict(test_features).flatten())

162/162 [==============================] - 0s 1ms/step


array([4.8572407, 4.856386 , 4.8566995, ..., 5.2472544, 5.2475   ,
       5.2472444], dtype=float32)

In [25]:
drive.mount("/content/gdrive")
model.save("/content/comp3000model")
zip_location = make_archive("model", "zip", "/content/comp3000model")
move(zip_location, "/content/gdrive/MyDrive/comp3000model/model.zip")
drive.flush_and_unmount()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
